In [1]:
in_file = "data/fvcom_riv_2014.nc"
comment = "Auto-generated from BDR's ssm-analysis notebooks"
out_file = "data/2014/ssm_riv.dat"

import numpy as np
from netCDF4 import Dataset

In [2]:
cdf = Dataset(in_file)
cdf

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    inflow_type: node
    point_st_type: calculated
    dimensions(sizes): time(365), node(201), siglay(10)
    variables(dimensions): float32 time(time), int32 node(node), float32 vqdist(node, siglay), float32 discharge(time, node), float32 temp(time, node), float32 salt(time, node)
    groups: 

In [4]:
siglayers = cdf.dimensions['siglay'].size
nodes = cdf.dimensions['node'].size
times = cdf.dimensions['time'].size

with open(out_file,"w") as f:
    print("{0}  {1} ! {2}".format(cdf.inflow_type, cdf.point_st_type, comment), file=f)
    print(nodes, file=f)
    for n in range(nodes):
        print("{:7d}".format(cdf['node'][n]), file=f)

    # VQDIST
    nodes_digits = int(np.ceil(np.log10(nodes+0.1)))
    siglay_formatstr = "{:%dd} " % nodes_digits + " ".join(["{:.4f}" for l in range(siglayers)])

    for n in range(nodes):
        print(siglay_formatstr.format(*[n+1] + list(cdf['vqdist'][n,:])), file=f)
        
    # Nqtime
    print("{:8d}".format(times), file=f)
    
    formatstr = "".join([" {:9.3e}" for l in range(nodes)])
    for t in range(times):
        print("{:9.2f}".format(cdf['time'][t]), file=f)
        for v in ('discharge','temp','salt'):
            print(formatstr.format(*cdf[v][t,:]), file=f)

In [5]:
cdf.close()